In [57]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
from PIL import Image
import os
import time

### Label mapping

In [14]:
class_names = ['buildings','forest','glacier','mountain','sea','street']
class_labels = {class_name:i for i, class_name in enumerate(class_names)}
print(class_labels)

number_classes = len(class_names)

IMAGE_SIZE = (150,150)

{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


### Data loading

In [69]:
train_dir = r"D:\Project\PG Project\Data\Image Data\seg_train\seg_train"
test_dir = r"D:\Project\PG Project\Data\Image Data\seg_test\seg_test"

def load_dataset():
    # create list of datasets
    datasets = [train_dir, test_dir]
    output = []
    
    for dataset in datasets:
        
        images1 = []
        labels1 = []
        print(f"loading {dataset}")
        
        for folder in os.listdir(dataset):
            # assign labels to each folder images
            label = class_labels[folder]
            for file in tqdm(os.listdir(os.path.join(dataset,folder))):
                image_path = os.path.join(os.path.join(dataset, folder), file)
                # read the image files stored in image_path
                image_file = cv2.imread(image_path)
                image_file = cv2.cvtColor(image_file, cv2.COLOR_BGR2RGB)
                image_file = cv2.resize(image_file, IMAGE_SIZE)
                
                images1.append(image_file)
                labels1.append(label)
                
        # convert the images and labels list to numpy array
        images1 = np.array(images1, dtype = 'float32')
        labels1 = np.array(labels1, dtype = 'int32')
        
        output.append((images1, labels1))
        print("Images file have been loaded")
                
    return output 

In [71]:
((train_images, train_labels), (test_images, test_labels)) = load_dataset()

loading D:\Project\PG Project\Data\Image Data\seg_train\seg_train


100%|█████████████████████████████████████████████████████████████████████████████| 2381/2381 [00:02<00:00, 884.37it/s]


Images file have been loaded
loading D:\Project\PG Project\Data\Image Data\seg_test\seg_test


100%|███████████████████████████████████████████████████████████████████████████████| 501/501 [00:02<00:00, 247.04it/s]


Images file have been loaded


In [73]:
print("train dataset size",len(train_images), len(train_labels))
print("test dataset size",len(test_images), len(test_labels))

train dataset size 13986 13986
test dataset size 2993 2993


### Normalisation

In [76]:
train_images_norm = train_images / 255.0
test_images_norm = test_images / 255.0

In [77]:
class EpochTimeLogger(Callback):
    def on_train_begin(self, logs=None):
        self.epoch_times = []

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start = time.time()

    def on_epoch_end(self, epoch, logs=None):
        duration = (time.time() - self.epoch_start) * 1000  # 毫秒
        self.epoch_times.append(duration)

### Model loading

In [79]:
VGG16_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [80]:
VGG16_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 150, 150, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 150, 150, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 75, 75, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 75, 75, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 75, 75, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 37, 37, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 37, 37, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 37, 37, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 18, 18, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 18, 18, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 18, 18, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 9, 9, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 9, 9, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 4, 4, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

### Top layers adding

In [86]:
x = VGG16_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(6, activation='softmax')(x)

In [88]:
for layer in  VGG16_model.layers:
    layer.trainable = False

In [90]:
model_vgg = Model(inputs=VGG16_model.input, outputs=predictions)

model_vgg.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [92]:
print(train_labels.shape)
print(train_labels[:5])

(13986,)
[0 0 0 0 0]


In [94]:
train_labels = to_categorical(train_labels, num_classes=6)  # shape → (N, 6)
test_labels = to_categorical(test_labels, num_classes=6)

earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

checkpointer = ModelCheckpoint(filepath='bestvalue_VGG.keras', verbose=0, save_best_only=True)

time_logger = EpochTimeLogger()

callback_list = [checkpointer, earlystopping, time_logger]

history_vgg = model_vgg.fit(
    train_images_norm, train_labels,
    validation_data=(test_images_norm, test_labels),
    batch_size=128,
    epochs=15,
    callbacks=callback_list
)


test_loss, test_acc = model_vgg.evaluate(test_images_norm, test_labels, verbose=0)
print("test loss and accuracy:", test_loss, test_acc)

Epoch 1/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 393s 4s/step - accuracy: 0.2359 - loss: 1.8382 - val_accuracy: 0.6752 - val_loss: 1.2919
Epoch 2/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 382s 3s/step - accuracy: 0.4937 - loss: 1.3160 - val_accuracy: 0.7514 - val_loss: 1.0024
Epoch 3/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 389s 4s/step - accuracy: 0.6294 - loss: 1.0471 - val_accuracy: 0.7832 - val_loss: 0.8365
Epoch 4/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 409s 4s/step - accuracy: 0.6904 - loss: 0.9029 - val_accuracy: 0.7975 - val_loss: 0.7317
Epoch 5/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 408s 4s/step - accuracy: 0.7378 - loss: 0.7874 - val_accuracy: 0.8086 - val_loss: 0.6622
Epoch 6/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 411s 4s/step - accuracy: 0.7483 - loss: 0.7297 - val_accuracy: 0.8136 - val_loss: 0.6133
Epoch 7/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 405s 4s/step - accuracy: 0.7621 - loss: 0.6814 - val_accuracy: 0.8219 - val_loss: 0.5769
Epoch 8/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 409s 4s/step - accuracy: 0.7730 - loss: 0.6485 - val_accu

### Training time

In [ ]:
for i, t in enumerate(time_logger.epoch_times):
    print(f"Epoch {i+1} duration: {t:.2f} ms")

In [ ]:
epoch_times = time_logger.epoch_times
epoch_numbers = np.arange(1, len(epoch_times) + 1)

plt.figure(figsize=(8, 5))
plt.plot(epoch_numbers, epoch_times, marker='o')
plt.title("Training Time per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Time (ms)")
plt.grid(True)
plt.xticks(epoch_numbers)
plt.tight_layout()
plt.show()

### Model accuracy

In [3]:
def plot_loss_accuracy(history):

    # create object of arrays of accuracy and loss
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    # number of epochs in our model
    epochs = range(1 ,len(acc) + 1)
    
    # call matplolib figure object and plot loss and accuracy curves
    plt.figure(figsize=(14,6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, marker='o', linestyle='-', color='royalblue', label='Training Accuracy')
    plt.plot(epochs, val_acc, marker='s', linestyle='--', color='darkorange', label='Validation Accuracy')
    plt.title("Training vs Validation Accuracy", fontsize=16)
    plt.xlabel('Epoch', fontsize=13)
    plt.ylabel('Accuracy', fontsize=13)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, marker='o', linestyle='-', color='crimson', label='Training Loss')
    plt.plot(epochs, val_loss, marker='s', linestyle='--', color='seagreen', label='Validation Loss')
    plt.title("Training vs Validation Loss", fontsize=16)
    plt.xlabel('Epoch', fontsize=13)
    plt.ylabel('Loss', fontsize=13)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

plot_loss_accuracy(history_vgg)

NameError: name 'history_vgg' is not defined

### Confusion matrix

In [ ]:
# 1. 获取模型预测结果（预测概率 → 预测类别）
predicted_probs = model_vgg.predict(test_images_norm)
predicted_labels = np.argmax(predicted_probs, axis=1)

# 2. 将 One-Hot 编码的真实标签还原为整数形式
true_labels = np.argmax(test_labels, axis=1)

# 3. 计算混淆矩阵
cm = confusion_matrix(true_labels, predicted_labels)

# 4. 设置类别标签
class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

# 5. 绘制混淆矩阵
fig, ax = plt.subplots(figsize=(7, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(ax=ax, cmap='Blues', colorbar=True, values_format='d')
plt.title("Confusion Matrix")
plt.grid(False)
plt.show()